<p style="font-size:32px; font-weight: bolder; text-align: center"> Structural representations <br/> for materials and molecules </p>
<p style="text-align: center"><i> authored by: <a href="mailto:michele.ceriotti@gmail.com"> Michele Ceriotti </a></i></p>

This notebook provides an overview of the ideas underlying the construction of (symmetric) descriptors of atomic structures, based on an atom-centered expansion of the neighbor density. 

We are going to work with a dataset of atomic structures - a snapshot of a simulation of undercooled iron - that allow us to introduce the concept of descriptors and to show how to apply some simple dimensionality reduction techniques to gather insights into the nature and behavior of different types of descriptors.

_References:_
- [Chem. Rev. 121, 9759 (2021)](https://pubs.acs.org/doi/10.1021/acs.chemrev.1c00021)
- [Phys. Rev. B 87, 184115 (2013)](http://link.aps.org/doi/10.1103/PhysRevB.87.184115)
- [J. Chem. Phys. 156, 204115 (2022)](https://aip.scitation.org/doi/10.1063/5.0087042)


### Packages and dependencies
This module uses `featomic` to compute descriptors, `chemiscope` to visualize structures and `sklearn` to perform principal component analysis.

In [ ]:
# scicode-widgets imports
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope

import ipywidgets
from ipywidgets import FloatSlider, IntSlider, Checkbox, Dropdown, HBox, Layout, HTML, Text

from markdown import markdown as mdwn

import scwidgets
from scwidgets.check import (
    Check,
    CheckRegistry,
    assert_numpy_allclose,
    assert_numpy_floating_sub_dtype,
    assert_shape,
    assert_type,
)
from scwidgets.code import ParametersPanel, CodeInput
from scwidgets.cue import CueObject, CueFigure
from scwidgets.exercise import CodeExercise, TextExercise, ExerciseRegistry

get_ipython().run_line_magic('matplotlib', 'widget')

In [ ]:
import numpy as np
import ase, ase.io
import itertools
from tqdm.notebook import tqdm

import featomic
from metatensor import mean_over_samples, Labels, TensorMap, TensorBlock, slice_block

from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV

### Answer settings

In [ ]:
exercise_registry = ExerciseRegistry(filename_prefix="module_02")
exercise_registry

In [ ]:
check_registry = CheckRegistry()
check_registry

In [ ]:
module_summary = TextExercise(
    description="""You can use this box to make general considerations, 
    or keep track of your doubts and questions about this notebook.""",
    exercise_registry=exercise_registry,
    title="Module comments",
    key="00"
)
display(module_summary)

<a id="data-driven"> </a>

# Descriptors of atomic environments in supercooled iron

As a first example we consider a structure which is cut ouf of a simulation of freezing iron ([Shibuta et al., Acta Mater. (2016)](https://www.sciencedirect.com/science/article/abs/pii/S1359645415301397)).
The snapshot contains a few solid nuclei embedded in a supercoled liquid.

We will use this structure to define atom-centered descriptors, and perform principal component analysis to color atoms based on whether they are in liquid or solid regions. 

Let's start by taking a look at the structure. Note that, to make the notebook fast enough, this is carved out of a larger structure, and so it is not periodic in the $x,y$ directions. 

In [ ]:
frame_iron = ase.io.read("data/iron-snapshot.xyz", 0)

crop_dz = 16 # selects a subset of the atoms to speed up and reduce memory requirements
crop_idx = np.where(
    (frame_iron.positions[:,2]<frame_iron.cell[2,2]/2+crop_dz/2) & 
    (frame_iron.positions[:,2]>frame_iron.cell[2,2]/2-crop_dz/2) 
)[0]
frame_iron = frame_iron[crop_idx]

core_dz = 4
max_cutoff = 6
core_idx = np.sort(np.where((frame_iron.positions[:,0]>max_cutoff+1) & (frame_iron.positions[:,0]<199-max_cutoff) &
                     (frame_iron.positions[:,1]>max_cutoff+1) & (frame_iron.positions[:,1]<199-max_cutoff) & 
                     (frame_iron.positions[:,2]<frame_iron.cell[2,2]/2+core_dz/2) & 
                     (frame_iron.positions[:,2]>frame_iron.cell[2,2]/2-core_dz/2) 
                    )[0])
frame_core = frame_iron[core_idx]

In [ ]:
# requires running in a jupyter notebook, and takes a while to load - it's > 100k atoms.
cs = chemiscope.show(frames=[frame_core], mode="structure", 
                     settings={"structure": [ {"bonds": False, "unitCell": True, 
                                               "spaceFilling": True
                             } ] },)
display(cs)

In [ ]:
# we compute now the neighbor list - might take a while
nl_code = featomic.NeighborList(cutoff=max_cutoff, full_neighbor_list=True)

#%%time
nl_all = nl_code.compute(frame_iron)

# this is a better way to select the relevant centers, exploiting the fact we are looking 
# for a single index match in an integer array
# extract int arrays
labs = core_idx
samp = np.asarray(nl_all.block(0).samples["first_atom"])
mask = np.zeros(len(samp), dtype=bool)

# sorting order and reverse order
sort_idx = np.argsort(samp)
sort_inv = np.argsort(sort_idx)
sort_samp = samp[sort_idx]

# now blocks with the same atom are contiguous so we can use searchsorted to find the bounds
lower = np.searchsorted(sort_samp,  labs)
upper = np.searchsorted(sort_samp,  labs+1)


for l,u in zip(lower, upper):
    mask[l:u] = True
mask = mask[sort_inv]

# actually we better extract a classical NL
neigh_dx = nl_all.block(0).values[sort_idx][mask].squeeze()
neigh_sz = (upper-lower)
neigh_i = np.cumsum(neigh_sz) -neigh_sz[0]
neigh_dr = np.sqrt((neigh_dx**2).sum(axis=1))

nl_all = None # free unneded mem

## Atom-centered environments

A first important consideration is that we are looking at an individual configuration, and that we want to identify atomic structures _within_ this configuration - distinguishing liquid regions, crystalline nuclei, and ideally the interfacial regions.

<center><img src="figures/environments.png" width="500"/></center>

One way to do this is to look at atomic _environments_ i.e. spherical atom-centered regions that we can describe in terms of the collection of interatomic distance vectors around each atom. You can look at the environments for the frame 

In [ ]:
# requires running in a jupyter notebook, and takes a while to load - it's > 100k atoms.
sel_env_idx = core_idx[np.array([8156, 2087, 4367, 6721, 4305, 1555])-1]
sel_env_idx.sort()
cs = chemiscope.show(frames=[frame_iron], mode="structure", 
                     properties={"index": list(range(len(sel_env_idx))), 
                                 "ref_idx": sel_env_idx},
                     settings={"structure": [ 
                         {"bonds": False, "unitCell": False, "spaceFilling": True,
                          "keepOrientation": True, 
                     'environments': {'activated': False, 'center': False}}] 
                              },  
                     environments=[[0,s,7.0] for s in sel_env_idx ] ,                     
                    )

In [ ]:
def update_co(code_exercise):
    cutoff = code_exercise.parameters["cutoff"]
    showenv = code_exercise.parameters["showenv"]
    cs.settings={"structure": [{"spaceFilling": True, "environments": {'activated': showenv, 
                                                 'center':showenv,
                                                 "cutoff":cutoff}}]}
cs_wp = ParametersPanel(
    showenv=Checkbox(value=False, description="show environments"),
    cutoff=FloatSlider(value=5.,min=2,max=max_cutoff,step=0.25, description=r"cutoff / Å"),    
)
cue_cs = CueObject()
with cue_cs:
    display(cs)
    
cs_demo = CodeExercise(
            parameters=cs_wp,
            outputs = cue_cs,
            update = update_co,
            update_mode="release")
cs_demo.run_update()
display(cs_demo)

In [ ]:
ex01_txt = TextExercise(
    description="""
It is always a good idea to take a good look at the data you are working with. 
Just play around with the viewer, look at the structure. 
What kind of features can you note by just observing the arrangement of the atoms?
Now switch on the environment view and use the atom slider to highlight a few select ones.
Can you easily recognize individual environments as liquid-like or solid-like?
    """,
    exercise_registry=exercise_registry,
    key="01",
    title="Exercise 01: What am I looking at?"
)
display(ex01_txt)

## Representations

Having taken the decision of focusing on atomic _environments_ for a structure $A$, that we will indicate as $A_i$, we need to come up with an appropriate way to encode information on the positions and types of _neighbors_ within the environment, $\{(a_j, \mathbf{r}_{ji})\}$.

<center><img src="figures/requirements.png" width="400"/></center>

In practice, we want to map to a vector of descriptors, or features $A_i\rightarrow\boldsymbol\xi(A_i)$. It is desirable to use a a mapping that fulfills a number of basic mathematical requirements: 

1. **Locality** (that is already satisfied by the use of atom-centered environments with a finite cutoff)
2. **Completeness** (two environments that are inequivalent should have different feature vectors)
3. **Smoothness** (the mapping between Cartesian coordinates and features should be differentiable, and "regular")
4. **Symmetry** (the mapping should be independent of rigid translations, rotations and permutation of atom indices

It is clear that $\{\mathbf{r}_{ij}\}$ fulfills (1) and (2), but is not smooth (the number of vectors change when atoms enter or leave the cutoff) and is only symmetric to translations. Using interatomic _distances_ $r_{ij}=|\mathbf{r}_{ij}|$ easily makes the features invariant to rotations, but are still dependent on the ordering of the atoms. 

Let's now try to build an invariant descriptor: a _histogram_ of the distances, discretized on a real-space grid. We use a kernel-density estimation, and include a _cutoff function_ to smoothly send contributions to zero as atoms approach the cutoff distance:

$$
\xi_k(A_i) = \sum_{j\in A_i} g(k - r_{ij}/\Delta_r) f_\mathrm{cut}(r_{ij})
$$

where $g(\cdot)$ is a Gaussian with zero mean and unit variance, and $\Delta_r=r_\mathrm{cut}/(n_\mathrm{grid}-1)$ is the resolution of the real-space grid, and 
$f_\mathrm{cut}(r_{ij})=(1+\cos \pi r_{ij}/r_\mathrm{cut})/2$.

In the following exercise you will be asked to implement this radial distribution fingerprint, and the exercise will compute and display it for the six environments visualized in the viewer for exercise 1. 

In [ ]:
def radial_fp(rij_list, rcut, ngrid):
    """
    compute a radial distribution fingerprint using a kernel density estimation in real-space
    
    
    :param rij_list: a list of interatomic distances for an environment
    :param rcut: cutoff distance
    :param ngrid: number of grid points and size of the feature vector
    
    :returns: a vector with the radial fistribution features computed for the given environment
    """
    import numpy as np
    rgrid = np.linspace(0, rcut, ngrid)
    feats = np.zeros(shape=rgrid.shape)
    
    ### ADD THE CALCULATION OF THE FEATURES HERE ###
    
    return feats

In [ ]:
# makes neighbor list for the six selected environments (ASE is too slow to be usable for this box)
px = frame_iron.positions
az = frame_iron.cell[2,2]
nl_idx = []
nl_dx = []
nl_dr = []
for isel in sel_env_idx:
    dx = px - px[isel]
    dx[:,2] /= az  # pbc along z
    dx[:,2] -= np.round(dx[:,2])
    dx[:,2] *= az
    dr = np.sqrt((dx**2).sum(axis=1))
    iw = np.where(dr<max_cutoff)[0]
    nl_idx.append(iw)
    nl_dx.append(dx[iw])
    nl_dr.append(dr[iw])

In [ ]:
ex02_img = mpl.image.imread('figures/selected-env.jpg')
def update_02(code_exercise):
    rcut, ngrid = code_exercise.parameters.values()
    ax, aximg = code_exercise.figure.get_axes()
    aximg.imshow(ex02_img)
    aximg.axis('off') 
    rgrid = np.linspace(0, rcut, ngrid)
    for dr, l in zip(nl_dr, ["A", "B", "C", "D", "E", "F"]):
        ygrid = code_exercise.code(dr, rcut, ngrid)    
        ax.plot(rgrid, ygrid,label=l)
    # ax.text(-4,8,f'$\ell = ${l:.3f}')
    ax.set_xlabel(r'$r$ / Å')
    ax.set_ylabel(r'$\xi$')
    ax.legend()

ex02_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    ngrid = IntSlider(value=10,min=5,max=20,description=r'$n_{grid}$') )

In [ ]:
ex02_figure, ex02_ax = plt.subplots(1, 2, figsize=(8,5), tight_layout=True)
ex02_ax[1].imshow(ex02_img)
ex02_ax[1].axis('off') 

ex02_code_demo = CodeExercise(
            code= radial_fp,
            parameters= ex02_pb,
            check_registry=check_registry,
            outputs = [ex02_figure],
            update = update_02,
    key="02",
    exercise_registry=exercise_registry,
    title="Exercise 02: Radial distribution fingerprints",
    description="""
Implement a function that computes a radial distribution fingerprint given the list
of distances for an environment, a cutoff and the number of grid points. 
You should implement the exact functional form given above, if you want checks to pass, 
but of course you're also encouraged to try something different!
"""
)

ex02_ref_input = [{"rij_list": np.array([1,3,4]), "rcut": 5, "ngrid": 10},
                 {"rij_list": np.array([5,7,8]), "rcut": 6, "ngrid": 4}
                 ]
ex02_ref_output = [(np.array([0.03542408, 0.47694048, 0.86904551, 0.21539186, 0.0558208 , 0.2951958, 
                              0.26366585 ,0.11845536, 0.05075245 ,0.00374063]),),
                  (np.array([1.29664583e-04, 7.22057763e-03, 6.38090792e-02, 1.96309381e-01]),)
                  ]

check_registry.add_check(ex02_code_demo,
    asserts=[
        assert_type,
        assert_shape,
        assert_numpy_allclose,
    ],
    inputs_parameters=ex02_ref_input,
    outputs_references=ex02_ref_output
)
                         
#                         inputs_parameters=ex_08_ref_input,
#                         reference_outputs = ex_08_ref_output,
#                         equal=ex08_chk,
#                        fingerprint=identity)

display(ex02_code_demo)

In [ ]:
ex02b_txt = TextExercise(
    description="""
Experiment with different grid resolutions, cutoff radius, etc. 
Can you recognize clear-cut differences between liquid-like and solid-like environments?
    """,
    exercise_registry=exercise_registry,
    key="02b",
    title="Exercise 02b: Resolving power of radial fingerprints."
)
display(ex02b_txt)

## Atom-centered symmetry functions

This set of radial features can be seen as a special case of so-called _atom-centered symmetry functions_ (ACSFs), one of the first types of representations used e.g. by [Behler and Parrinello](http://doi.org/10.1103/PhysRevLett.98.146401). 


<center><img src="figures/radial-acsf.png" width="500"/><br/>
<i> Representative examples of radial symmetry functions.</i><br/><br/>
</center>


ACSFs are designed as bespoke functions $\phi_k$ of the internal coordinates of the environment, accumulated over neighbors to achieve invariance to atom index permutations.
They can be generalized to also include functions of distances and angles (3-body symmetry functions) and can be tuned to focus on the structural features that are most discriminating, or most straightforwardly related to the structure-property relations one is trying to learn.
Radial (two-body) symmetry functions take the form

$$
\xi_k(A_i) = \sum_{j\in A_i} \phi_k(r_{ij}) f_\mathrm{cut}(r_{ij})
$$

where $\phi_k$ has typically a parametric form, or enumerates a set of orthogonal basis functions. 

In [ ]:
def radial_acsf(rij_list, rcut, delta, rs):
    """
    compute a radial distribution fingerprint using a kernel density estimation in real-space
    
    
    :param rij_list: a list of interatomic distances for an environment
    :param rcut: cutoff distance
    :param delta: the smearing of the Gaussian ACSF
    :param rs: the center of the Gaussian ACSF
    
    :returns: a float containing the value of the ACSF for the environment
    """
    import numpy as np
    
    acsf = 0.0
    ### ADD THE CALCULATION OF THE ACSF VALUE HERE ###
    
    return acsf

In [ ]:
ex03_img = mpl.image.imread('figures/selected-env.jpg')
def update_03(code_exercise):
    rcut, delta, rs = code_exercise.parameters.values()
    ax, aximg = code_exercise.figure.get_axes()
    aximg.imshow(ex02_img)
    aximg.axis('off') 
    rgrid = np.linspace(0, rcut, 100)
    ygrid = np.zeros_like(rgrid)
    for ir, r in enumerate(rgrid):
        ygrid[ir] = code_exercise.code([r], rcut, delta, rs)    
    ax.plot(rgrid, ygrid, 'r-')
    ax.set_xlabel(r'$r$ / Å')
    ax.set_ylabel(r'$\phi_k(r)$')

    labels = []
    for dr, l in zip(nl_dr, ["A", "B", "C", "D", "E", "F"]):
        acf = code_exercise.code(dr, rcut, delta, rs) 
        labels.append(f"{l}: {acf:9.4f}")
    aximg.legend(handles=[mpl.patches.Patch(color="w", )]*6, labels=labels,
                 handlelength=0.1, loc='lower left')

ex03_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    delta = FloatSlider(value=0.5,min=0.1,max=2,step=0.1,description=r'$\Delta$ / Å'),
    rs = FloatSlider(value=5,min=2,max=6,step=0.1,description=r'$r_s$ / Å'),
    )

In [ ]:
ex03_figure, ex03_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex03_ax[1].imshow(ex03_img)
ex03_ax[1].axis('off') 

ex03_code_demo = CodeExercise(
            code= radial_acsf,
            parameters= ex03_pb,
            check_registry=check_registry,
            outputs = [ex03_figure],
            update = update_03,
    update_mode="manual",
    key="03",
    exercise_registry=exercise_registry,
    title="Exercise 03: Radial ACSF",
    description=mdwn(r"""
Implement a function that computes a Behler-Parrinello atom-center symmetry function
of the form

$$
\phi_k(r) = \exp\[-(r-r_s)^2/\delta^2\]  f_{\matrhm{cut}(r)
$$

using a cosine cutoff function.
""")
)

ex03_ref_input = [{"rij_list": np.array([1,3,4]), "rcut": 5, "delta": 1, "rs":5},
                 {"rij_list": np.array([5,3,8]), "rcut": 6, "delta": 0.5, "rs":4}
                 ]
ex03_ref_output = [(0.04145736008495549,),
                  (0.010384734606641187,)
                  ]

check_registry.add_check(ex03_code_demo,
    asserts=[
        assert_type,
        assert_shape,
        assert_numpy_allclose,
    ],
    inputs_parameters=ex03_ref_input,
    outputs_references=ex03_ref_output
)
                         

display(ex03_code_demo)

In [ ]:
ex03b_txt = TextExercise(
    description="""
Observe how the shape of the symmetry function, and its value for the 
various environments, change with its parameters. Try to find values that maximise the difference
between solid-like and liquid-like environments. 
    """,
    exercise_registry=exercise_registry,
    key="03b",
    title=""
)
display(ex03b_txt)

## Discretized density expansion

The neighbor density provides a way to characterize the position of atoms in the vicinity of a tagged center. It is instructive to see it built up starting from a structure-level smooth atom density, in which a structure  $A$ is represented in terms of localized functions (e.g. Gaussians) centered on each atom $i$, "labelled" by their chemical nature $a$

$$
\rho_A^a(\mathbf{x}) = \langle a \mathbf{x} | A; \rho\rangle = \sum_{i \in A} \delta_{a a_i} \langle \mathbf{x} | \mathbf{r}_i \rangle.
$$

Note how, by summing over $i$, the identity of atoms of the same species is lost, making the representation invariant to atom labeling. 

We use  the notation $\langle \mathbf{x} | \mathbf{r}_i \rangle = g(\mathbf{x}-\mathbf{r}_i)$ to emphasize how the full structure is built as a sum of terms that describe individual atoms, and how this description can be implemented in any continuous or discrete basis. 

In general terms, in analogy with the Dirac notation used to describe a quantum state, we use  $\langle q | A\rangle$ to indicate a descriptor $| A\rangle$ for an entity $A$, discretized in a basis that is enumerated by the index $q$. 
See Section 3.1 of [this review](https://doi.org/10.1021/acs.chemrev.1c00021) for a gentler introduction. 

This density is then symmetrized with respect to translations (reflecting the fact that atomic properties are invariant to rigid translations of a molecule) which leads to expressing the structure descriptors as a sum of descriptors of _atom centered environments_ $A_i$,


$$
\langle a \mathbf{x} | A; \rho\rangle = \sum_i \langle a \mathbf{x} | \rho_i\rangle
$$

$$
\langle a \mathbf{x} | \rho_i\rangle = \sum_{j \in A_i} \delta_{a a_j} \langle \mathbf{x} | \mathbf{r}_{ji} \rangle.
$$

where the Gaussians are evaluated at the interatomic distance vectors $\mathbf{r}_{ji}=\mathbf{r}_j-\mathbf{r}_i$.

To manipulate this atom-centered density, it is more convenient to express it on a discrete basis. Guided by symmetry considerations, and in analogy with what is done routinely in quantum chemistry for the electron wafefunction (or density) we use a basis of radial functions $R_{nl}(x) \equiv \langle x|nl\rangle$ and [spherical harmonics](https://en.wikipedia.org/wiki/Spherical_harmonics) $Y^m_{l}(\hat{\mathbf{x}}) \equiv \langle \hat{\mathbf{x}}|lm\rangle$

$$
\langle a nlm | \rho_i\rangle = \int \mathrm{d}\mathbf{x} 
 \langle nl| x\rangle  \langle lm| \hat{\mathbf{x}} \rangle
\langle a \mathbf{x} | \rho_i\rangle  
$$

If you are confused by this rather abstract notation, you can also define the neighbor density as an explicit sum over neigbor positions $\{\mathbf{r}_{ij}\}$,

$$
\langle a nlm | \rho_i\rangle \equiv \rho_{nlm}^a(A_i) = \sum_{j\in A_i} Y^m_l(\hat{\mathbf{r}}_{ij}) \tilde{R}_{nl}(r_{ij}),
$$

where the Gaussian smearing of the density has been implemented as a transformation of the radial basis. See [Goscinski et al.](http://doi.org/10.1063/5.0057229) for a derivation of the equivalence of the two expressions. 

In [ ]:
def density_expansion(frame, rcut, nmax, lmax, sigma, select_idx):
    """
    compute a discretization of the neighbor density in terms 
    of radial functions and spherical harmonics. a Gaussian smearing
    is applied to the neighbor density
    
    :param frame: ase.Atoms frame to compute
    :param rcut: cutoff distance
    :param nmax: number of radial functions
    :param lmax: maximum angular momentum
    :param sigma: Gaussian smearing of the density
    :param select_idx: indices for the atoms to use as centers
    
    :returns: a TensorMap containing the density expansion coefficients
    """
    from featomic import SphericalExpansion
    from metatensor import Labels
    
    # parameters of the density expansion
    hypers = {
        "cutoff": { "radius": rcut,
                    "smoothing": { "type": "ShiftedCosine", "width": 0.5},
                    },
        "density": {"type": "Gaussian", "width": sigma },
        "basis": { "type": "TensorProduct", "max_angular": lmax, 
            "radial": { "type": "Gto", "max_radial": nmax } }
    }
    
    calculator = SphericalExpansion(**hypers)
    
    if select_idx is None:
        rhoi = calculator.compute(frame)
    else:
        rhoi = calculator.compute(frame,
                selected_samples=Labels(names="atom", values=select_idx.reshape(-1,1))
        )
    
    # Uncomment to print some of the metadata associated with the density coefficients
    # print(rhoi)
    # print(rhoi.block(1))
    
    return rhoi

In [ ]:
ex04_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    nmax = IntSlider(value=4,min=1,max=8,description=r'$n_{max}$'),
    lmax = IntSlider(value=2,min=1,max=6,description=r'$l_{max}$'),
    sigma = FloatSlider(value=0.5,min=0.3,max=1,step=0.1,description=r'$\sigma$ / Å'),
    environment = Dropdown(value="A", options=["A", "B", "C", "D", "E", "F"],
                          description="environment")
    )

ex04_img = mpl.image.imread('figures/selected-env.jpg')
def combine_l(tmap, i_env):
    feats=[]
    for b in tmap:
        feats.append(b.values[i_env])
    return np.vstack(feats)

ex04_cache_pars = (-1, -1, -1, -1, "")
ex04_cache_value = None
ex04_cbar = None
ex04_cbar_ax = None
def update_04(code_exercise):
    global ex04_cache_pars, ex04_cache_value, ex04_cbar, ex04_cbar_ax

    # Unpack parameters
    rcut, nmax, lmax, sigma, env = code_exercise.parameters.values()
    fig = code_exercise.figure
    ax, aximg = fig.get_axes()[:2]

    # Show background image
    aximg.imshow(ex04_img)
    aximg.axis('off')

    # Use cache if parameters & code haven't changed
    current_pars = (rcut, nmax, lmax, sigma, code_exercise.code.function_body)
    if ex04_cache_pars == current_pars:
        rhoi = ex04_cache_value
    else:
        rhoi = code_exercise.code(frame_iron, rcut, nmax, lmax, sigma, sel_env_idx)
        ex04_cache_value = rhoi
        ex04_cache_pars = current_pars

    # Get selected environment index
    envidx = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5}
    feats = combine_l(rhoi, envidx[env])

    # Normalize and visualize features
    frange = np.max(np.abs(feats))
    norm = mpl.colors.SymLogNorm(vmin=-frange, vmax=frange, linthresh=1e-1)
    cax = ax.matshow(feats.T, cmap='seismic', norm=norm)

    # Set up a fixed colorbar axis (once)
    if ex04_cbar_ax is None:
        from mpl_toolkits.axes_grid1 import make_axes_locatable
        divider = make_axes_locatable(ax)
        ex04_cbar_ax = divider.append_axes("bottom", size="5%", pad=0.4)

    # Draw colorbar into that fixed axis
    ex04_cbar = fig.colorbar(cax, cax=ex04_cbar_ax, orientation='horizontal')

    # Axis labels
    ax.set_ylabel("n")
    ax.set_xlabel("(l,m)")

    # Mark angular momentum blocks
    xticklabels = []
    xtickpos = []
    for l in range(lmax + 1):
        ax.add_patch(mpl.patches.Rectangle(
            (-0.5 + l**2, -0.5), 2*l + 1, nmax + 1,
            edgecolor='black', facecolor='none', linewidth=3
        ))
        xticklabels.append(f"$l={l}$")
        xtickpos.append(l**2 + l)

    ax.set_xticks(xtickpos)
    ax.set_xticklabels(xticklabels)


In [ ]:
ex04_figure, ex04_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex04_ax[1].imshow(ex04_img)
ex04_ax[1].axis('off') 

ex04_code_demo = CodeExercise(
            code= density_expansion,
            parameters= ex04_pb,            
            outputs = [ex04_figure],
            update = update_04,
    update_mode="release",
    key="04",
    exercise_registry=exercise_registry,
    title="Exercise 04: Density expansion",
    description=mdwn("""
This exercise is set up to compute neighbor density expansion coefficients, based
on the specified parameters. You can observe how the density coefficients change 
for different hyperparameters. You can also change other options that are not 
provided as function parameters, but make sure to revert them to the initial values
as this function is also used in what follows.

_NB:_ It takes a few seconds to update the figure when you change the hypers, but 
it should be faster to switch between environments, as the features are computed for
all environments and cached. 
""")
)

display(ex04_code_demo)
ex04_code_demo.run_update()

In [ ]:
rhoi = ex04_code_demo.code(frame_iron, 6.0, 4, 2, 0.5, sel_env_idx)

`featomic` calculators return a `metatensor.TensorMap` object, that works as a container that holds blocks of data. The pattern is reminiscent of a `dict`, but with some more structure and metadata: each block is associated with a _key_, which consists in a tuple of ints. The set of keys is a `metatensor.Labels` object, that also keeps track of the _names_ that describe each index in the key. 

The expansion keys hold the `o3_lambda` and `o3_sigma` indices that correspond to the angular momentum rotational symmetry and the symmetry with respect to inversion ($1$ being associated with the "normal" symmetry of a spherical harmonic. The two following indices correspond to the atomic number of the central atom `center_type` and of the neighbors `neighbor_type` - here 26 for both as we are dealing with pure iron.  

For instance, if you compute a `TensorMap` that contains density coefficients, `rhoi`, you can print an overview of the map content

```python
print(rhoi)
```

In [ ]:
print(rhoi)

<center><img src="figures/mtt-tensorblock-components.svg" width="300"/><br/>
<i> Structure of a TensorBlock</i><br/><br/>
</center>

Each entry in a `TensorMap` is a `TensorBlock` object, that contains a dense storage of properties, with three axes that identify the `samples` the properties refer to, their `components` (e.g. the $m$ index in a spherical harmonic of order $l$) and the actual `properties`, that may be associated with further indices that enumerate all possible values (e.g. the radial basis index $n$). 

If you print out one of the blocks, you will get an overview of the associated metadata, e.g.

```python
print(rhoi).block(1)
```

In [ ]:
print(rhoi.block(1))

You can see the [metatensor documentation](http://docs.metatensor.org) if you want to find out how to manipulate and access the entries in a `TensorMap` or `TensorBlock`. We will see a few examples further down. 

## Three-body correlations: SOAP features

The $l=0$ part of the density-expansion coefficients $\langle an00|\rho_i\rangle$ corresponds to a discretization of the pair correlation function: using a real-space basis,

$$
    \langle ax00|\rho_i\rangle \approx \sum_{j\in A_i} \delta_{a a_j} \langle x | r_{ji} \rangle 
$$

where $ \langle x | r_{ji} \rangle $ is a localized function centered on $r_{ji}$. 

In order to obtain a richer description of the atomic environment it is possible to combine several copies of $\langle a\mathbf{x} | \rho_i \rangle$, to build $\nu$-neighbors atom-centered density correlations (ACDCs). 
The formalism we use was introduced by [Willatt et al.](https://doi.org/10.1063/1.5090481), and is explained in detail, discussing its relation with the leading frameworks for atomistic machine learning, in a [review by Musil et al.](https://doi.org/10.1021/acs.chemrev.1c00021)

Essentially, the idea is that considering tensor products of the atom density provides simultaneous information on the mutual position of several neighbors

$$
\langle \mathbf{x} |  \rho_i \rangle \langle \mathbf{x}' |  \rho_i \rangle =
\sum_{jj'\in A_i}
\langle \mathbf{x} |\mathbf{r}_{ji} \rangle \langle \mathbf{x}' |\mathbf{r}_{j'i} \rangle. 
$$

The invariant part of this two-neighbor correlation function can be extracted by taking a symmetrized product of the density coefficients,
$$
\langle aa'nn'l|\overline{\rho_i^{\otimes 2}}\rangle \propto 
\sum_m  \langle anlm|\rho_i\rangle \langle a'n'lm|\rho_i\rangle
$$

These are the [SOAP powerspectrum coefficients](http://doi.org/10.1103/PhysRevB.87.184115), that have been widely used in machine-learning models and especially in the context of Gaussian approximation potentials - kernel models based on SOAP features (see [Deringer et al. 2021](http://doi.org/10.1021/acs.chemrev.1c00022) for a review of kernel methods in the field). 

For those confused by the Dirac notation, this can be written using the vectorial notation for the density descriptors, $\rho_{nlm}^a(A_i)$: 

$$
p_{nn'l}^{aa'}(A_i) \propto \sum_m \rho_{nlm}^a(A_i)\rho_{n'lm}^{a'}(A_i)
$$

Using the [addition theorem for spherical harmonics](https://mathworld.wolfram.com/SphericalHarmonicAdditionTheorem.html) it is possible to draw a connection between the SOAP powerspectrum and three-body symmetry functions computed as a sum over pairs of neighbors. Ignoring for simplicity the element indices, 

$$
\langle nn'l|\overline{\rho_i^{\otimes 2}}\rangle \propto
\sum_{jj'\in A_i} \tilde{R}_{nl}(r_{ij}) \tilde{R}_{nl}(r_{ij'}) P_l(\hat{\mathbf{r}}_{ij}\cdot \hat{\mathbf{r}}_{ij'})
$$

One sees that computing SOAP features from the density expansion avoids the double sum over neighbors, at the cost of computing a large number of $m$ components of the spherical harmonics - an idea often referred to as the _density trick_.

`featomic` provides the infrastructure to evaluate SOAP features using the `SoapPowerSpectrum` calculator. You can see that the syntax is very similar to that for the density expansion features, as those define the discretization level on top of which the power spectrum is computed. 

In [ ]:
def soap_ps(frame, rcut, nmax, lmax, sigma, select_idx):
    """
    compute the SOAP powerspectrum using the featomic calculator
    
    :param frame: ase.Atoms frame to compute
    :param rcut: cutoff distance
    :param nmax: number of radial functions
    :param lmax: maximum angular momentum
    :param sigma: Gaussian smearing of the density
    :param select_idx: indices for the atoms to use as centers
    
    :returns: a TensorMap containing the SOAP powerspectrum coefficients
    """
    from featomic import SoapPowerSpectrum
    from metatensor import Labels
    
    # parameters of the density expansion
    hypers = {
        "cutoff": { "radius": rcut,
                    "smoothing": { "type": "ShiftedCosine", "width": 0.5},
                    },
        "density": {"type": "Gaussian", "width": sigma },
        "basis": { "type": "TensorProduct", "max_angular": lmax, 
            "radial": { "type": "Gto", "max_radial": nmax } }
    }
    
    calculator = SoapPowerSpectrum(**hypers)
    
    if select_idx is None:
        soap = calculator.compute(frame)
    else:
        soap = calculator.compute(frame,
                selected_samples=Labels(names="atom", values=select_idx.reshape(-1,1))
        )
    
    # Uncomment to print some of the metadata associated with the density coefficients
    # print(soap)
    # print(soap.block(0))
    
    return soap

In [ ]:
ex05_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    nmax = IntSlider(value=4,min=1,max=8,description=r'$n_{max}$'),
    lmax = IntSlider(value=2,min=1,max=6,description=r'$l_{max}$'),
    sigma = FloatSlider(value=0.5,min=0.3,max=1,step=0.1,description=r'$\sigma$ / Å'),
    environment = Dropdown(value="A", options=["A", "B", "C", "D", "E", "F"],
                          description="environment")
    )

ex05_img = mpl.image.imread('figures/selected-env.jpg')

ex05_cache_pars = (-1, -1, -1, -1, "")
ex05_cache_value = None
ex05_cbar = None

def update_05(code_exercise):
    global ex05_cache_pars, ex05_cache_value, ex05_cbar
    rcut, nmax, lmax, sigma, env = code_exercise.parameters.values()
    fig = code_exercise.figure
    ax, aximg = fig.get_axes()[:2]
    aximg.imshow(ex05_img)
    aximg.axis('off') 

    if ex05_cache_pars == (rcut, nmax, lmax, sigma, code_exercise.code.function_body):
        soap = ex05_cache_value
    else:
        soap = code_exercise.code(frame_iron, rcut, nmax, lmax, sigma, sel_env_idx)
        ex05_cache_value = soap
        ex05_cache_pars = (rcut, nmax, lmax, sigma, code_exercise.code.function_body)
    envidx={"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}
    feats = soap.block(0).values.squeeze()[envidx[env]].reshape(-1, nmax+1)
    frange = np.max(np.abs(feats))
    
    norm = mpl.colors.SymLogNorm(vmin=-frange, vmax=frange, linthresh=1e-3)
    
    cax=ax.matshow(feats.T, cmap='seismic', norm=norm)
    if ex05_cbar is None:
        ex05_cbar = fig.colorbar(cax, ax=ax, orientation='horizontal' )
    else:
        ex05_cbar.update_normal(cax)
    ax.set_ylabel("n2")
    ax.set_xlabel("(l,n1)")
    xticklabels = []
    xtickpos = []
    for l in range((lmax+1)):
        ax.add_patch(mpl.patches.Rectangle(
            (-0.5+l*(nmax+1),-0.5), (l+1)*(nmax+1), nmax+1,
            edgecolor='black', facecolor='none', linewidth=3
        ))
        xticklabels.append(f"$l={l}$")
        xtickpos.append((l)*(nmax+1))
    ax.set_xticks(xtickpos); ax.set_xticklabels(xticklabels)

In [ ]:
ex05_figure, ex05_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex05_ax[1].imshow(ex05_img)
ex05_ax[1].axis('off') 

ex05_code_demo = CodeExercise(
            code= soap_ps,
            parameters= ex05_pb,            
            outputs = [ex05_figure],
            update = update_05,
    update_mode="release",
    key="05",
    exercise_registry=exercise_registry,
    title="Exercise 05: SOAP features",
    description=mdwn("""
This exercise is set up to compute SOAP power spectrum coefficients, based
on the specified parameters. You can observe how the values change for different
environments and hyperparameters. You can also change other options that are not 
provided as function parameters, but make sure to revert them to the initial values
as this function is also used in what follows.

_NB:_ It takes a few seconds to update the figure when you change the hypers, but 
it should be faster to switch between environments, as the features are computed for
all environments and cached. Note also the very large difference in magnitude for the
$l=0$ and $l>0$ blocks. 
""")
)

display(ex05_code_demo)
ex05_code_demo.run_update()

It is also instructive to compute the powerspectrum manually. 
This requires a bit of bookkeeping, in particular to create
`metatensor` objects. Note also the normalization factor, that is
omitted in the definitions above, but is important to match
the `featomic`-computed values. You can check the [documentation of
numpy.einsum](XXXXXXXX) if you're not familiar with the function. 

In [ ]:
def soap_manual(frame, rcut, nmax, lmax, sigma, select_idx, density_function):
    """
    compute the SOAP powerspectrum manually, starting from the density 
    expansion coefficients 
    
    :param frame: ase.Atoms frame to compute
    :param rcut: cutoff distance
    :param nmax: number of radial functions
    :param lmax: maximum angular momentum
    :param sigma: Gaussian smearing of the density
    :param select_idx: indices for the atoms to use as centers
    :param density_function: function that computes the density coefficients,
        with signature density_function(frame, rcut, nmax, lmax, sigma, select_idx)
    
    :returns: a TensorBlock containing the SOAP powerspectrum
    """
    import numpy as np
    from metatensor import TensorBlock, Labels
    
    rhoi = density_function(frame, rcut, nmax, lmax, sigma, select_idx)
    
    soap_values = []
    soap_props = []
    for key, block in rhoi.items():    
        # loops over the blocks of the density expansion, corresponding to the l components.
        # each block is indexed by (sample, angular_m, radial_n)
        soap_values.append( # these are the actual SOAP values
                             np.einsum('XXXX',  # enter the correct einsum string here
                               block.values, block.values
                             ).reshape(len(block.samples),-1)*
                             (-1)**key['o3_lambda']/np.sqrt(2*key['o3_lambda']+1) )
        # these are the corresponding l,n,n' indices that are accumulated for convenience
        soap_props.append(  [ [key["o3_lambda"], n, p] for n in block.properties['n'] for p in block.properties['n']] )
    
    # combines the values 
    soap_values = np.hstack(soap_values)
    soap_props = np.vstack(soap_props)
    soap_samples = rhoi.block(0).samples
    soap_manual = TensorBlock(values=soap_values, 
                              properties=Labels(["l", "n1", "n2"], soap_props), 
                              components=[],
                              samples=soap_samples) 
                              
    return soap_manual

In [ ]:
ex06_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    nmax = IntSlider(value=4,min=1,max=8,description=r'$n_{max}$'),
    lmax = IntSlider(value=2,min=1,max=6,description=r'$l_{max}$'),
    sigma = FloatSlider(value=0.5,min=0.3,max=1,step=0.1,description=r'$\sigma$ / Å'),
    environment = Dropdown(value="A", options=["A", "B", "C", "D", "E", "F"],
                          description="environment")
    )

ex06_img = mpl.image.imread('figures/selected-env.jpg')

ex06_cache_pars = (-1, -1, -1, -1, "")
ex06_cache_value = None
ex06_cbar = None

def update_06(code_exercise):
    global ex06_cache_pars, ex06_cache_value, ex06_cbar
    rcut, nmax, lmax, sigma, env = code_exercise.parameters.values()
    fig = code_exercise.outputs[0].figure
    ax, aximg = fig.get_axes()[:2]
    aximg.imshow(ex06_img)
    aximg.axis('off') 

    if ex06_cache_pars == (rcut, nmax, lmax, sigma, code_exercise.code.function_body):
        soap = ex06_cache_value
    else:
        soap = code_exercise.code(frame_iron, rcut, nmax, lmax, 
                                              sigma, sel_env_idx, ex04_code_demo.code)
        ex06_cache_value = soap
        ex06_cache_pars = (rcut, nmax, lmax, sigma, code_exercise.code.function_body)
    envidx={"A":0, "B":1, "C":2, "D":3, "E":4, "F":5}
    feats = soap.values.squeeze()[envidx[env]].reshape(-1, nmax+1)
    frange = np.max(np.abs(feats))
    
    norm = mpl.colors.SymLogNorm(vmin=-frange, vmax=frange, linthresh=1e-3)
    
    cax=ax.matshow(feats.T, cmap='seismic', norm=norm)
    if ex06_cbar is None:
        ex06_cbar = fig.colorbar(cax, ax=ax, orientation='horizontal' )
    else:
        ex06_cbar.update_normal(cax)
    ax.set_ylabel("n2")
    ax.set_xlabel("(l,n1)")
    xticklabels = []
    xtickpos = []
    for l in range((lmax+1)):
        ax.add_patch(mpl.patches.Rectangle(
            (-0.5+l*(nmax+1),-0.5), (l+1)*(nmax+1), (nmax+1),
            edgecolor='black', facecolor='none', linewidth=3
        ))
        xticklabels.append(f"$l={l}$")
        xtickpos.append((l+0.5)*(nmax+1))
    ax.set_xticks(xtickpos); ax.set_xticklabels(xticklabels)

In [ ]:
ex06_figure, ex06_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex06_ax[1].imshow(ex06_img)
ex06_ax[1].axis('off') 

ex06_code_demo = CodeExercise(
            code= soap_manual,
            parameters= ex06_pb,            
            outputs = [ex06_figure],
            update = update_06,
    update_mode="release",
    key="06",
    exercise_registry=exercise_registry,
    title="Exercise 06: Manual evaluation of SOAP features",
    description=mdwn("""
This exercise is set up to compute SOAP power spectrum coefficients _manually_,
starting from the density coefficients (in practice these are computed using the
function in exercise 4. All the bookkeeping and `metatensor` code is already 
implemented, you only need to write the correct `einsum` string to evaluate the 
summation (that should return for each "l" block the features in the order 
`(sample, n1, n2)`.
""")
)

display(ex06_code_demo)

# Automatic identification of environments

In this section we try to come up with a strategy to differentiate between solid-like and liquid-like regions. This is not easy, because the snapshot is taken from a finite-temperature simulation, and even in the solid parts the atomic positions are free to fluctuate. 
We will look at plots for a thin slie within the sample, to keep the cost of computing descriptors (that use the Python functions you defined previously) as low as possible. 

## Manually tuning a descriptor

A first way to define a structural desciptor is to come up with one based on intuition, or trial-and-error. For example, one could take an ACSF, and optimize its parameters to find some that have a high discriminating power. As we shall see, this is not necessarily an easy task!

In [ ]:
ex07_figure, ex07_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex07_output = CueFigure(ex07_figure)

ex07_xy = frame_iron.positions[core_idx,:2]
ex07_cbar = None
def update_07(code_exercise):
    global ex07_cbar
    rcut, delta, rs = code_exercise.parameters.values()
    fig = code_exercise.outputs[0].figure
        
    ax = fig.get_axes()[0]
    ax.axis('off')
    vals = np.zeros(len(core_idx))
    acsf=ex03_code_demo.code
    for i in tqdm(range(len(vals))):
        vals[i] = acsf(neigh_dr[neigh_i[i]:neigh_i[i]+neigh_sz[i]], delta=delta, rs=rs, rcut=rcut)
    cax=ax.scatter(ex07_xy[:,0], ex07_xy[:,1], c=vals, marker='.', s=5,
                  vmin=vals.min(), vmax = vals.max() )       
    
    if ex07_cbar is None:
        ex07_cbar = fig.colorbar(cax, ax=ax )
    else:
        ex07_cbar.update_normal(cax)

    ax = fig.get_axes()[1]
    ax.hist(vals, color='red', bins=50)
    ax.set_xlabel(r"$\xi$")
    ax.set_ylabel(r"counts")

ex07_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    delta = FloatSlider(value=0.5,min=0.1,max=2,step=0.1,description=r'$\Delta$ / Å'),
    rs = FloatSlider(value=5,min=3,max=8,step=0.1,description=r'$r_s$ / Å'),
    )

ex07_code_demo = CodeExercise(
    parameters= ex07_pb,
    outputs = [ex07_output],
    update = update_07,
    update_mode="manual",
    #key="07",
    title="Exercise 07: Radial ACSF as a structural fingerprint",
    description=mdwn("""
Play around with the ACSF parameters (this exercise uses the function *you* implemented
in exercise 03), and see if you can find values that differentiate clearly between liquid-like
and solid-like environments.
"""
))

In [ ]:
display(ex07_code_demo)
ex07_code_demo.run_update()

In [ ]:
ex07b_txt = TextExercise(
    description="""
Observe the variability in the descriptor values.
Why do you think it's hard to find good values for the ACSF to make it good at discriminating?
Think also at the radial function plots in exercise 02.
    """,
    exercise_registry=exercise_registry,
    key="07b",
    title=""
)
display(ex07b_txt)

## Principal component analysis

We could then try to compute many symmetry functions (or equivalently to use the radial descriptors that are discretized on a grid). We would however than have the problem of having a large number of fingerprints for each environment: how to then use a single value to identify the two phases? Luckily, we can use a principal component analysis to identify different modes!

In [ ]:
ex08_wci = CodeInput(
        function_name="features_pca", 
        function_parameters="xi_list, n_components",
        docstring="""
        perform a princpal component analysis of a list of environment descriptors
        
        
        :param xi_list: a list of features computed for some atomic environments
        :param n_components: the number of PCA components to evaluate
        
        :returns: the list of PCA features for the given environments
""",
        function_body="""
# here you only have to call sklearn functions, see ex. 01 or the sklearn documentation
# if you forgot how to perform PCA. for simplicity (and given there are way more environments
# than descriptors and components) you should apply the transformation to the same features
# so you can also use fit_transform. 

pca = xi_list
return pca
"""
        )

In [ ]:
ex08_figure, ex08_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex08_output = CueFigure(ex08_figure)

ex08_xy = frame_iron.positions[core_idx,:2]
ex08_cbar = None
ex08_cache_pars=(-1,-1,"")
ex08_cache_vals=None
def update_08(code_exercise):
    global ex08_cbar, ex08_cache_pars, ex08_cache_vals
    rcut, ngrid, ipca = code_exercise.parameters.values()
    fig = code_exercise.outputs[0].figure

    # cache values if we only change ipca
    if (ex08_cache_pars[0] == rcut and 
        ex08_cache_pars[1] == ngrid and
        ex08_cache_pars[2] == ex02_code_demo.code.function_body):
        vals = ex08_cache_vals
    else:
        vals = np.zeros((len(core_idx), ngrid))
        radial=ex02_code_demo.code
        for i in tqdm(range(len(vals))):
            vals[i] = radial(neigh_dr[neigh_i[i]:neigh_i[i]+neigh_sz[i]], rcut=rcut, ngrid=ngrid)
        ex08_cache_vals  = vals
        ex08_cache_pars = (rcut, ngrid, ex02_code_demo.code.function_body)
        
    ax = fig.get_axes()[0]
    ax.axis('off')
    
    pcafull = code_exercise.code(vals, n_components=8)
    pca = pcafull[:,ipca]
    cax=ax.scatter(ex08_xy[:,0], ex08_xy[:,1], c=pca, marker='.', s=5,
                  vmin=pca.min(), vmax = pca.max() )       
    
    if ex08_cbar is None:
        ex08_cbar = fig.colorbar(cax, ax=ax )
    else:
        ex08_cbar.update_normal(cax)

    ax = fig.get_axes()[1]
    ax.hist(pca, color='red', bins=50)
    ax.set_xlabel(r"PCA")
    ax.set_ylabel(r"counts")

ex08_pb =  ParametersPanel(
    rcut = FloatSlider(value=5,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    ngrid = IntSlider(value=8,min=4,max=32,step=1,description=r'$n_{grid}$'),
    ipca = IntSlider(value=0,min=0,max=6,step=1,description=r'$i_{PCA}$')
    )
                         
ex08_code_demo = CodeExercise(
    code=ex08_wci,
    check_registry=check_registry,
    parameters= ex08_pb,
    outputs = [ex08_output],
    update = update_08,
    update_mode="manual",
    key="08",
    exercise_registry=exercise_registry,
    title="Exercise 08: Feature PCA",
    description=mdwn("""
Write a function that computes the principal component analysis for a set of 
descriptors. The visualizer will then compute radial features using the code you
wrote in exercise 01, with the parameters specified, and plot the color-coded
structure and a histogram of the projected features. You can select which PCA
component you display.

_NB:_ It can take up to a minute to compute all the radial descriptors, but the 
viewer will cache the values so as long as you only change the PCA index the 
plotting will be almost instantaneous. This function will also be used in 
some of the following exercises so make sure it works correctly!
"""
))

In [ ]:

ex08_ref_input = [{"xi_list": np.array([[1,3,4],[2,5,7],[12,-5,7]]), "n_components":2},
                 {"xi_list": np.array([[11,32,4],[23,3,-7],[23,5,7],[12,-5,7]]), "n_components": 3}
                 ]
ex08_ref_output = [(np.array([[-4.52278769, -1.88265544],
                              [-4.7507803 ,  1.85204928],
                              [ 9.27356799,  0.03060617]]),),
 (np.array([[ 24.04316249 ,  1.42811158 , -1.03650133],
            [ -7.06128363 , -10.24872457 , -2.50787124],
            [ -4.63236046 ,  0.28829947 ,  6.60648318],
            [-12.3495184  ,  8.53231352  , -3.0621106 ]]),)]

check_registry.add_check(ex08_code_demo,
    asserts=[
        assert_type,
        assert_shape,
        assert_numpy_allclose,
    ],
    inputs_parameters=ex08_ref_input,
    outputs_references=ex08_ref_output
)

In [ ]:
display(ex08_code_demo)

In [ ]:
ex08b_txt = TextExercise(
    description="""
Observe the range spanned by the different components. How does it change with increasing index?
Why? Is the first fingerprint the best at distinguishing solid and liquid?
    """,
    exercise_registry=exercise_registry,
    key="08b",
    title=""
)
display(ex08b_txt)

In [ ]:
ex08c_txt = TextExercise(
    description="""
Can you get much better results than with the ACSF-based approach? 
The difference between solid and liquid regions should be
clearly reflected in the color-coded structure, and
an ideal fingerprint should show clearly a bimodal distribution.
    """,
    exercise_registry=exercise_registry,
    key="08c",
    title=""
)
display(ex08c_txt)

## Incorporating directional information with density descriptors

If the problem is that the radial descriptors do not have enough resolution to identify solid structures, it makes sense to use features that are aware of the directional information such as the density descriptors. Given the large number of descriptors available, we also perform a PCA to select the most informative combinations. Note that this section uses the functions you have introduced above in exercises 4 and 8, to compute the density descriptors and to perform the PCA. 

In [ ]:
def combine_l_all(tmap):
    feats=[]
    for b in tmap:
        feats.append(b.values.reshape((len(b.values),-1)) )
    return np.hstack(feats)

In [ ]:
ex09_figure, ex09_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex09_output = CueFigure(ex09_figure)

ex09_xy = frame_iron.positions[core_idx,:2]
ex09_cbar = None
ex09_cache_pars=(-1,-1,"")
ex09_cache_vals=None
rhoi_pca = None
def update_09(code_exercise):
    global ex09_cbar, ex09_cache_pars, ex09_cache_vals, rhoi_pca
    rcut, nmax, lmax, sigma, ipca = code_exercise.parameters.values()
    fig = code_exercise.outputs[0].figure

    # cache values if we only change ipca
    if (ex09_cache_pars == (rcut, nmax, lmax, sigma, ex04_code_demo.code.function_body) ):
        vals = ex09_cache_vals
    else:        
        dsel = len(core_idx)//8 +1
        vals = [] 
        for i in tqdm(range(8)): # compute in pieces so we can show progress
            rhoi = ex04_code_demo.code(frame_iron, rcut, nmax, lmax, sigma, core_idx[i*dsel:(i+1)*dsel])
            vals.append(combine_l_all(rhoi))        
        vals = np.vstack(vals)
        ex09_cache_vals  = vals
        ex09_cache_pars = (rcut, nmax, lmax, sigma, ex04_code_demo.code.function_body)
        
    ax = fig.get_axes()[0]
    ax.axis('off')
    
    rhoi_pca = ex08_wci.get_function_object()(vals, n_components=8)
    pca = rhoi_pca[:,ipca]
    cax=ax.scatter(ex09_xy[:,0], ex09_xy[:,1], c=pca, marker='.', s=5,
                  vmin=pca.min(), vmax = pca.max() )       
    
    if ex09_cbar is None:
        ex09_cbar = fig.colorbar(cax, ax=ax )
    else:
        ex09_cbar.update_normal(cax)

    ax = fig.get_axes()[1]
    ax.hist(pca, color='red', bins=50)
    ax.set_xlabel(r"PCA")
    ax.set_ylabel(r"counts")

ex09_pb =  ParametersPanel(
    rcut = FloatSlider(value=4,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    nmax = IntSlider(value=4,min=1,max=8,description=r'$n_{max}$'),
    lmax = IntSlider(value=6,min=1,max=8,description=r'$l_{max}$'),
    sigma = FloatSlider(value=0.5,min=0.3,max=1,step=0.1,description=r'$\sigma$ / Å'),
    ipca = IntSlider(value=0,min=0,max=6,step=1,description=r'$i_{PCA}$')
    )
                         
ex09_code_demo = CodeExercise(
    parameters= ex09_pb,
    outputs = [ex09_output],
    update = update_09,
    update_mode="manual",
    #key="09",
    #exercise_registry=exercise_registry,
    title="Exercise 09: Density expansion PCA",
    description=mdwn(""" Experiment with the parameters of the density expansion 
    to see if you can identify more clearly the solid regions. 
Note that it takes up to a minute to recompute descriptors after you change the parameters 
so be patient. 

_Hint:_ You'll probably need to include high values of $l$. Given there is lots of 
directional information, low-cutoff values are usually sufficient (and even beneficial)""")
)

In [ ]:
display(ex09_code_demo)

In [ ]:
ex09b_txt = TextExercise(
    key="09b",
    exercise_registry=exercise_registry,
    title="",
    description=mdwn("""
Note the values of the PCA features for the different solid regions. 
Do they differ? Why? We will investigate this further in the next exercises.
""") )
    
display(ex09b_txt)

## Invariant descriptors of the atomic structure

We can do the same thing for SOAP descriptors

In [ ]:
ex10_figure, ex10_ax = plt.subplots(1, 2, figsize=(8,4), tight_layout=True)
ex10_output = CueFigure(ex10_figure)

ex10_xy = frame_iron.positions[core_idx,:2]
ex10_cbar = None
ex10_cache_pars=(-1,-1,-1,-1,"")
ex10_cache_vals=None
soap_pca = None
def update_10(code_exercise):
    global ex10_cbar, ex10_cache_pars, ex10_cache_vals, soap_pca
    rcut, nmax, lmax, sigma, select_l, ipca = code_exercise.parameters.values()
    fig = code_exercise.outputs[0].figure

    # cache values if we only change ipca
    if (ex10_cache_pars == (rcut, nmax, lmax, sigma, ex05_code_demo.code.function_body) ):
        vals = ex10_cache_vals
    else:        
        dsel = len(core_idx)//8 +1
        vals = [] 
        for i in tqdm(range(8)): # compute in pieces so we can show progress
            soap = ex05_code_demo.code(frame_iron, rcut, nmax, lmax, sigma, core_idx[i*dsel:(i+1)*dsel])
            vals.append(soap.block(0).values.squeeze())        
        vals = np.vstack(vals)
        ex10_cache_vals  = vals
        ex10_cache_pars = (rcut, nmax, lmax, sigma, ex05_code_demo.code.function_body)
        
    ax = fig.get_axes()[0]
    ax.axis('off')
    npca = 8
    if select_l!="": # picks manually some l channels
        select_l = np.array(list(map(int, select_l.split(','))))
        if select_l.max()>lmax:
            raise ValueError("Cannot extract l channels larger than lmax")
        select_blocks = []
        nmax2=(nmax+1)**2
        for l in select_l:
            # Extend the result list by adding a range starting from 'start' and continuing for 'nmax' elements
            select_blocks.extend(range(l*nmax2, (l+1)*nmax2))
        vals=vals[:,select_blocks]
        if len(select_blocks)<npca:
            npca = len(select_blocks)

    soap_pca = ex08_wci.get_function_object()(vals, n_components=8)
    pca = soap_pca[:,ipca]
    cax=ax.scatter(ex10_xy[:,0], ex10_xy[:,1], c=pca, marker='.', s=5,
                  vmin=pca.min(), vmax = pca.max() )       
    
    if ex10_cbar is None:
        ex10_cbar = fig.colorbar(cax, ax=ax )
    else:
        ex10_cbar.update_normal(cax)

    ax = fig.get_axes()[1]
    ax.hist(pca, color='red', bins=50)
    ax.set_xlabel(r"PCA")
    ax.set_ylabel(r"counts")

ex10_pb =  ParametersPanel(
    rcut = FloatSlider(value=4,min=3,max=max_cutoff,step=0.1,description=r'$r_{cut}$ / Å'),
    nmax = IntSlider(value=4,min=1,max=8,description=r'$n_{max}$'),
    lmax = IntSlider(value=6,min=1,max=8,description=r'$l_{max}$'),
    sigma = FloatSlider(value=0.5,min=0.3,max=1,step=0.1,description=r'$\sigma$ / Å'),
    select_l = Text(value="",description=r'$l$ channels'),
    ipca = IntSlider(value=0,min=0,max=6,step=1,description=r'$i_{PCA}$')    
    )
                         
ex10_code_demo = CodeExercise(
    parameters= ex10_pb,
    outputs = [ex10_output],
    update = update_10,
    update_mode="manual",
    #key="10",
    #exercise_registry=exercise_registry,
    title="Exercise 10: SOAP PCA",
    description=mdwn("""Experiment with the hyperparameters to improve the contrast
    between solid and liquid regions. 
Note that it takes up to a minute to recompute descriptors after you change the parameters 
so be patient. 

_Hint:_ Start from values that gave good resolution for the density expansion. 
You will notice less clear-cut classification. One way to improve resolution is 
to select only specific $l$ channels that have good resolving power (because they 
are sensitive to fcc ordering). You can pick a comma-separated list of values, e.g. 
`5,7,8`.""")
)

In [ ]:
display(ex10_code_demo)

In [ ]:
ex10b_txt = TextExercise(
    key="10b",
    exercise_registry=exercise_registry,
    title="",
    description=mdwn("""
Compare the values of the PCA features in the various solid nuclei.
Are there qualitative differences with what we saw for density expansion coefficients?
In which modeling or analysis use cases would you employ either type of descriptors?
""") )
    
display(ex10b_txt)

## Combined visualization

Only _after_ having completed all the exercises, and having found good parameters to visualize the phase transition, click the `Update` button below. It will generate an interactive viewer that you can use to get further insights on the relation between the structure and the SOAP and density based PCA descriptors.

In [ ]:
def update_cs(code_exercise):
    frame_slice = frame_iron[core_idx]
    chemiscope.write_input("module_02-iron_analysis.chemiscope.json.gz", frames=[frame_slice], 
                     properties={
                         "SOAP-PCA": soap_pca[:,:4],
                          "RHO-PCA": rhoi_pca[:,:4]
                     },
                     settings={
                         'map': {'x': {'property': 'RHO-PCA[1]'},  
                                 'y': {'property': 'RHO-PCA[2]'},
                                 'color': {'property': 'SOAP-PCA[1]'},
                          'palette': 'viridis'},
                         "structure": [ 
                         {"bonds": False, "unitCell": False,
                           'spaceFilling': True,
                          "keepOrientation": True,  
                          'color': {'property': 'SOAP-PCA[1]','palette': 'viridis'},                         
                     'environments': {'activated': False, 'center': False}}] 
                              },  
                     environments=chemiscope.all_atomic_environments(frame_slice) ,                     
                    )
    with cue_cs_final:
        display(chemiscope.show_input("module_02-iron_analysis.chemiscope.json.gz"))

cs_wp_final = ParametersPanel(
    showenv=Checkbox(value=False, description="show environments"),
    cutoff=FloatSlider(value=5.,min=2,max=max_cutoff,step=0.25, description=r"cutoff / Å"),    
)

cue_cs_final = CueObject()
    
cs_demo_final = CodeExercise(
            #parameters=cs_wp_final,
            outputs = cue_cs_final,
            update = update_cs,
            update_mode="manual")

display(cs_demo_final)

[Download chemiscope datafile]("module_02-iron_analysis.chemiscope.json.gz")